# PPT_ACHR_2

In [1]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import scipy
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import yaml
import re

import hypnogram as hp
from ecephys.utils import replace_outliers
import ecephys.plot as eplt
import ecephys.signal.timefrequency as tfr
import ecephys.signal.kd_utils as kd
import ecephys.signal.kd_plotting as kp
import ecephys.signal.kd_pax as kpx
import tdt_xarray as tx

bands_def = dict(delta=(0.5, 4.1), theta=(4.1, 8), beta = (13, 20), low_gamma = (40, 55), high_gamma = (65, 80))

kd_ref = {}
kd_ref['echans'] = [1,2,3]
kd_ref['fchans']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
kd_ref['analysis_root'] = Path(r"L:\Data\ACHR_PROJECT_MATERIALS\PPT_ACHR_1\analysis_data")
kd_ref['tank_root'] = Path(r"L:\Data\PPT_ACHR_1_TANK")

# Test-a1

In [2]:
a1p = "L:\Data\ACHR_2_TANK\ACHR_2-test-a1"

In [3]:
a2a1 = {}
a2a1['er'], a2a1['er_spg'] = kd.get_data_spg(a1p, store='EEGr', t1=0, t2=0, channel=kd_ref['echans'])
a2a1['e'], a2a1['e_spg'] = kd.get_data_spg(a1p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
a2a1['f'], a2a1['f_spg'] = kd.get_data_spg(a1p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

FileNotFoundError: [Errno 2] No such file or directory: 'L:\\Data\\ACHR_2_TANK\\ACHR_2-test-a1/'